<a href="https://colab.research.google.com/github/pysepty/DCF/blob/master/monitoring_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install yfinance
!pip install finvizfinance

In [2]:
import pandas as pd
import yfinance as yf

In [3]:
benchmark = pd.DataFrame({'Symbol':['SPY','NDX']})

In [ ]:
# growing_conviction = pd.DataFrame({'Symbol':['INMD','MYRG','ORGO','DGX','NIU','IRBT','EXPI']})
# market_leaders = pd.DataFrame({'Symbol':['CSCO','ADBE','NVDA','CRM','INTU','ACN','LLY','DHR',
#                                          'COST','HD','MRNA']})
# winners_win_list = pd.DataFrame({'Symbol':['ISRG','A','REGN','CROX','DGX','NUE','TX',
#                                            'PRFT','GTLS','ALKS']})
# find_100_bagger = pd.DataFrame({'Symbol':['LLY','NVO','PFPT',
#                                           'ASAN','SE','OPEN','SPLK','PATH']})
# undervalue_growth_watchlist = pd.DataFrame({'Symbol':['EBAY','AMKR','CROX','WSM','WGO',
#                                                       'EBS','HOLX','REGN',
#                                                       'LPX','NUE','TX',]})
# intheradar_watchlist = pd.DataFrame({'Symbol':['ABNB','SNOW','PINS','JMIA','AMPG','SFT','ZEN','GRUB','PTON','WISH','PLTR','ETSY','DOCN','MGNI','GKOS',
#                                                'UCTT','ON','TSM','IRBT','ATEN','CRM','TTD','APPS','TECH',
#                                                'CODX','DMTK','TRNS','NVTA','IRWD','JNJ','CARA','SMG','SUPN']})
# renewable_watchlist = pd.DataFrame({'Symbol':['GTLS','PLUG','ENPH','DQ','TSLA','NIO','LAZR','CHPT','BLNK']})
# stock_list = pd.concat([benchmark, market_leaders, winners_win_list, growing_conviction, find_100_bagger, undervalue_growth_watchlist, 
#                         intheradar_watchlist]).drop_duplicates().reset_index(drop=True)
#stock_list = pd.concat([benchmark, market_leaders, growing_conviction]).drop_duplicates().reset_index(drop=True)

In [5]:
#@title Define function fundamental
from finvizfinance.quote import finvizfinance

def desc(symbol):
    stock = finvizfinance(symbol)
    stock_fundament = stock.TickerFundament()
    name = stock_fundament['Company']
    sector = stock_fundament['Sector']
    industry = stock_fundament['Industry']
    mcap = stock_fundament['Market Cap']    
    return name, sector, industry, mcap

def dcf(symbol):
    stock = finvizfinance(symbol)
    stock_fundament = stock.TickerFundament()
    current = float(stock_fundament['Price'])    
    mcap = stock_fundament['Market Cap']
    gross = stock_fundament['Gross Margin']
    volat = float(stock_fundament['Volatility W'].strip('%'))
    try:
        ins = float(stock_fundament['Insider Own'].strip('%'))    
    except:
        ins = float('NaN')
    
    try:
        ROE = float(stock_fundament['ROE'].strip('%'))
    except:
        ROE = float('NaN')
        
    try:
        peg = float(stock_fundament['PEG'])
    except:
        peg = float('NaN')
        
    try:
        wst = float(stock_fundament['Target Price'])
        buffer = round(100*(wst-current)/current, 2)
    except:
        wst = float('NaN')
        buffer = float('NaN')
        
    try:
        eps_ttm = float(stock_fundament['EPS (ttm)'])                        
        
        if eps_ttm > 0:        
            y5n  = float(stock_fundament['EPS next 5Y'].strip('%'))
            y5p  = float(stock_fundament['EPS past 5Y'].strip('%'))            
            ge  = (y5n+y5p)/4
            yr = 5
            dr = 10
            tg = 2.5
 
            eps = [eps_ttm]
            disc =[1]
            disc_eps = [eps_ttm]
            for year in range(1, yr+1):
                eps.append(eps[year - 1] * (1 + ge/100))
                disc.append(1 / (1 + (dr / 100))**(year))
                disc_eps.append(eps[year]*disc[year])  
 
            terminal_value = ((eps[-1] * (1 + tg/100)) / (dr/100 - tg/100))
            eps.append(terminal_value)
            disc_eps.append(eps[year+1]*(1 / (1 + (dr / 100))**(year)))
 
            PV = sum(disc_eps)
            #print("-> Expected value based on EPS growth in 5Y: ${}".format(round(PV)))
            
            fair    = round(PV,2)
            margin  = 100*(fair-current)/current
            return volat, ins, gross, ROE, peg, current, wst, buffer, fair, margin, mcap
        else:
            return volat, ins, gross, ROE, peg, current, wst, buffer, float('NaN'), float('NaN'), mcap
    except:
        return volat, ins, gross, ROE, peg, current, wst, buffer, float('NaN'), float('NaN'), mcap

In [6]:
#@title Define function Return and Sharpe ratio
#6mo = 126days ago
#3mo = 63days ago
#1mo = 22days ago
def return_period(symbol,since_days_ago):
    start = (len(df_latest_price) - since_days_ago) - 1
    
    if start>=0:
        cum_return = df_latest_price[symbol]/df_latest_price.iloc[start][symbol]
        stock_return = 100 * (cum_return[-1] -1)
        return round(stock_return,1)
    else:
        return float('NaN')
    
def sharpe_ratio(symbol,since_days_ago):  
    start = (len(df_latest_price) - since_days_ago) - 1
    
    if start>=0:
        daily_return = df_latest_price.iloc[start:][symbol].pct_change(1)
        SR  = daily_return.mean()/daily_return.std()
        ASR = (252**0.5)*SR
        return round(ASR,1)
    else:
        return float('NaN')

In [12]:
# xlsfile = "/content/drive/MyDrive/Colab Notebooks/active_list.xlsx"
# xlsfile = "/content/drive/MyDrive/Colab Notebooks/EverlastingStocks.xlsx"
# xlsfile = "/content/drive/MyDrive/Colab Notebooks/list.xlsx"
xlsfile = "/content/drive/MyDrive/Colab Notebooks/data/portfolio1.xls"
data = pd.read_excel(xlsfile, skiprows=3)
stock_list = pd.DataFrame(data[['Symbol']])
stock_list = pd.concat([benchmark,stock_list]).drop_duplicates().reset_index(drop=True)

#stock_list.head()
data.head()

,Symbol,Cost/share,Qty
0,JCI,72.006,5
1,ETSY,184.240,1
2,AZEK,38.730,2
3,FUV,9.930,5
4,FTCI,8.550,6


In [ ]:
# stock_list[stock_list['Symbol']=='ILIKF']

In [13]:
# some stocks are not available in finviz
delete_stock = ['AUR','HNHPF','ILIKF','SMRT','XONE','KVSB']
stock_list = stock_list.set_index('Symbol').drop(delete_stock).reset_index()

In [14]:
#@title get sector information
#stock_desc = stock_list.copy()
stock_list[['name', 'sector', 'industry', 'market cap']] = stock_list[2:].apply(lambda x: desc(x['Symbol']), axis = 1, result_type = 'expand')
# (stock_desc[2:].set_index('Symbol').sort_values(by=['Symbol']).style)

pd.set_option('display.max_rows', stock_list.shape[0]+1)
import ipywidgets as widgets
from IPython.display import display
# stock_desc.sort_values(by="Symbol")

col = stock_list.columns.to_list()
col.insert(0, 'ALL')
dropdown_column = widgets.Dropdown(options = col)
output = widgets.Output()
def dropdown_column_eventhandler(change):
    output.clear_output()
    with output:
        if (change.new == 'ALL'):
          display(stock_list)
        else:
          display(stock_list.sort_values(by=change.new))          
dropdown_column.observe(dropdown_column_eventhandler, names='value')
display(dropdown_column)
display(output)

Dropdown(options=('ALL', 'Symbol', 'name', 'sector', 'industry', 'market cap'), value='ALL')

Output()

In [15]:
#@title Get historical data from yahoo
tickerStrings = list(stock_list['Symbol'])
df_list = list()
df_latest_price = pd.DataFrame()
for ticker in tickerStrings:
    data = yf.download(ticker, auto_adjust=True, progress = False, period='6mo')
    #data['ticker'] = ticker  # add this column becasue the dataframe doesn't contain a column with the ticker
    df_latest_price[ticker] = data['Close']
 
#df_latest_price.tail()

In [16]:
#@title Run calculation Return and Sharpe Ratio
stock_list['7D performance %'] = stock_list.apply(lambda x: return_period(x['Symbol'], 6), axis=1)
stock_list['15D performance %'] = stock_list.apply(lambda x: return_period(x['Symbol'], 14), axis=1)
stock_list['30D performance %'] = stock_list.apply(lambda x: return_period(x['Symbol'], 29), axis=1)
stock_list['3MO performance %'] = stock_list.apply(lambda x: return_period(x['Symbol'], 63), axis=1)
stock_list['6MO performance %'] = stock_list.apply(lambda x: return_period(x['Symbol'], 125), axis=1)
 
stock_list['7D SR'] = stock_list.apply(lambda x: sharpe_ratio(x['Symbol'], 6), axis=1)
stock_list['15D SR'] = stock_list.apply(lambda x: sharpe_ratio(x['Symbol'], 14), axis=1)
stock_list['30D SR'] = stock_list.apply(lambda x: sharpe_ratio(x['Symbol'], 29), axis=1)
stock_list['3MO SR'] = stock_list.apply(lambda x: sharpe_ratio(x['Symbol'], 63), axis=1)
stock_list['6MO SR'] = stock_list.apply(lambda x: sharpe_ratio(x['Symbol'], 125), axis=1)

In [17]:
#@title Run fundamental
stock_list[['Volatility W %', 'Insider Own %', 'gross', 'ROE %', 'PEG', 'Latest Price $', 'WST latest target $', 'Buffer %', 'Fair $', 'Margin of Safety %', 'market cap']] = stock_list[2:].apply(lambda x: dcf(x['Symbol']), axis = 1, result_type = 'expand')

In [18]:
#@title Define style for the table
 
def green_background_performance(series):
    highlight = 'background-color: palegreen;'
    default = ''
    return [highlight if e > series.loc['SPY']+5 else default for e in series]   
 
def green_background_sharpe(series):
    highlight = 'background-color: palegreen;'
    default = ''
    return [highlight if e > series.loc['SPY']+1 else default for e in series]  
 
def red_background(series):
    highlight = 'background-color: tomato;'
    default = ''
    return [highlight if e < series.loc['SPY'] else default for e in series] 
 
def bold_max_value_in_series(series):
    highlight = 'font-weight: bold;'
    default = ''
    return [highlight if e == series.max() else default for e in series]  
 
def undervalue(cell_value):
    highlight = 'background-color: palegreen;'
    default = ''
    if cell_value > 30:
        return highlight
    else:
        return default  
 
def insider(cell_value):
    highlight = 'background-color: palegreen;'
    default = ''
    if cell_value >= 5:
        return highlight
    else:
        return default  
 
def ROE_highlight(cell_value):
    highlight_green = 'background-color: palegreen;'
    highlight_red = 'background-color: tomato;'
    default = ''
    if cell_value > 25:
        return highlight_green
    elif cell_value < 15:
        return highlight_red
    else:
        return default  
    
def peg_highlight(cell_value):
    highlight_green = 'background-color: palegreen;'
    highlight_red = 'background-color: tomato;'
    default = ''
    if cell_value < 2:
        return highlight_green
    elif cell_value > 4:
        return highlight_red
    else:
        return default  
    
def volatility(cell_value):
    highlight_green = 'background-color: palegreen;'
    highlight_red = 'background-color: tomato;'
    default = ''
    if cell_value < 2:
        return highlight_green
    elif cell_value > 4:
        return highlight_red
    else:
        return default  
 
def red_background_negative_values(cell_value):
    highlight = 'background-color: tomato;'
    default = ''
    if cell_value < 5:
        return highlight
    else:
        return default     

def table_style(df):
  performanceCL = ['7D performance %', '15D performance %', '30D performance %', '3MO performance %', '6MO performance %']
  sharpeCL = ['7D SR','15D SR','30D SR','3MO SR','6MO SR']
  numCL = performanceCL + sharpeCL + ['Volatility W %', 'Insider Own %', 'ROE %', 'PEG', 
                                      'Latest Price $', 'WST latest target $', 'Fair $', 'Margin of Safety %']
  return (df.set_index('Symbol').style.format('{:.1f}',subset=numCL)
  .apply(green_background_performance, axis=0, subset=performanceCL)
  .apply(green_background_sharpe, axis=0, subset=sharpeCL)
  .apply(red_background, axis=0, subset=performanceCL+sharpeCL)
  .apply(bold_max_value_in_series, axis=0, subset=performanceCL)
  .applymap(insider, subset=['Insider Own %'])
  .applymap(ROE_highlight, subset=['ROE %'])
  .applymap(undervalue, subset=['Margin of Safety %', 'Buffer %'])
  .applymap(red_background_negative_values, subset=['Margin of Safety %', 'Buffer %'])
  .applymap(volatility, subset=['Volatility W %'])
  .applymap(peg_highlight, subset=['PEG']))



In [19]:
#@title display current performance

import ipywidgets as widgets
from IPython.display import display

sector_list = stock_list.sector.unique().tolist()
sector_list.pop(0)
sector_list.insert(0, 'ALL')
dropdown_sector = widgets.Dropdown(options = sector_list)
dropdown_analysis = widgets.Dropdown(options = stock_list.columns.to_list(), value="7D performance %")
dropdown_analysis_type = widgets.Dropdown(options = ["descending","ascending"])
output_analysis = widgets.Output()

def filter_sector_sorted(sector_selected,kolom,jenis_sorting):
  output_analysis.clear_output()
  with output_analysis:
    if (sector_selected == 'ALL'):
      display_list = stock_list.copy()
    else:
      display_list = stock_list.copy()
      display_list = display_list[display_list.sector==sector_selected]
      display_list = display_list.append(stock_list[stock_list.Symbol=="SPY"])
    if (jenis_sorting == "descending"):
      display(table_style(display_list.sort_values(by=kolom, ascending=False)))
    else:
      display(table_style(display_list.sort_values(by=kolom, ascending=True)))

def dropdown_sector_eventhandler(change):
  filter_sector_sorted(change.new, dropdown_analysis.value, dropdown_analysis_type.value)
def dropdown_analysis_eventhandler(change):
  filter_sector_sorted(dropdown_sector.value, change.new, dropdown_analysis_type.value)
def dropdown_analysis_type_eventhandler(change):
  filter_sector_sorted(dropdown_sector.value, dropdown_analysis.value, change.new)

dropdown_sector.observe(dropdown_sector_eventhandler, names='value')    
dropdown_analysis.observe(dropdown_analysis_eventhandler, names='value')
dropdown_analysis_type.observe(dropdown_analysis_type_eventhandler, names='value')

display(dropdown_sector)
display(dropdown_analysis)
display(dropdown_analysis_type)
display(output_analysis)

Dropdown(options=('ALL', 'Industrials', 'Consumer Cyclical', 'Technology', 'Healthcare', 'Communication Servic…

Dropdown(index=5, options=('Symbol', 'name', 'sector', 'industry', 'market cap', '7D performance %', '15D perf…

Dropdown(options=('descending', 'ascending'), value='descending')

Output()

# biotech
##### ORGO : high ROE, 95% owned by insiders, big potential based on WST, and fair value estimation
##### REGN : high ROE, stable price movement

# Health care equipment
##### INMD : high ROE, low PEG

# Life Sciences Tools and Services
##### A : stable price movement

# Cloud computing stocks
#### 

# Crypto
#### BTBT
